## Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point

## Data Cleaning

In [2]:
# Load the data and inspect the first few rows
df = pd.read_csv('../data/raw/311_Service_Requests_2024_2025_reduced.csv.bz2',
                 compression = 'bz2', low_memory = False)
df.head()

,Created Date,Closed Date,Agency Name,Complaint Type,Location Type,Incident Zip,City,Borough,Latitude,Longitude
0,12/31/2024 11:59:38 PM,01/01/2025 12:26:35 AM,New York City Police Department,Illegal Fireworks,Street/Sidewalk,11218,NaN,BROOKLYN,40.640915,-73.973642
1,12/31/2024 11:59:33 PM,01/02/2025 05:08:17 PM,New York City Police Department,Noise - Residential,Residential Building/House,10466,BRONX,BRONX,40.891872,-73.860168
2,12/31/2024 11:59:32 PM,01/01/2025 12:18:51 AM,New York City Police Department,Noise - Residential,Residential Building/House,11221,BROOKLYN,BROOKLYN,40.688335,-73.930144
3,12/31/2024 11:59:31 PM,01/01/2025 09:01:36 AM,New York City Police Department,Noise - Residential,Residential Building/House,10466,BRONX,BRONX,40.891872,-73.860168
4,12/31/2024 11:59:21 PM,01/01/2025 12:42:47 AM,New York City Police Department,Noise - Residential,Residential Building/House,11230,BROOKLYN,BROOKLYN,40.634745,-73.964936


In [3]:
# Normalize column names to snake case
df.columns = ['_'.join(column.split(' ')).lower() for column in df.columns]

In [4]:
# Inspect column types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3458311 entries, 0 to 3458310
Data columns (total 10 columns):
 #   Column          Dtype  
---  ------          -----  
 0   created_date    object 
 1   closed_date     object 
 2   agency_name     object 
 3   complaint_type  object 
 4   location_type   object 
 5   incident_zip    object 
 6   city            object 
 7   borough         object 
 8   latitude        float64
 9   longitude       float64
dtypes: float64(2), object(8)
memory usage: 263.8+ MB


In [5]:
# Check missingness of columns
df.isnull().sum()

created_date           0
closed_date       107983
agency_name            0
complaint_type         0
location_type     416849
incident_zip       34384
city              161561
borough                0
latitude           53606
longitude          53606
dtype: int64

### Dealing with Time Columns

In [6]:
# Convert date columns to datetime types
df['created_date'] = pd.to_datetime(df['created_date'], format = "%m/%d/%Y %I:%M:%S %p")
df['closed_date'] = pd.to_datetime(df['closed_date'], format = "%m/%d/%Y %I:%M:%S %p")

# Replace NaT in closed date with last datetime of 2024
df['closed_date'] = df['closed_date'].fillna(pd.Timestamp("2024-12-31 23:59:59"))

# Truncate data to only include data where the incident occurred and resolved within 2024
df = df[(df['created_date'] >= '2024-01-01') & (df['created_date'] < '2025-01-01')]
df = df[(df['closed_date'] >= '2024-01-01') & (df['closed_date'] < '2025-01-01')]

In [7]:
# Create time of day category based on hour of incident
# Extract hour
hours = df['created_date'].dt.hour

# Define bins and labels
# 12AM-5:59AM = Night, 6AM-11:59AM = Morning, 12:00PM-5:59PM = Afternoon, 6:00PM-11:59PM = Night
bins = [0, 6, 12, 18, 24]
labels = ['Night', 'Morning', 'Afternoon', 'Evening']

# Create a new column with time-of-day category
df['time_of_day'] = pd.cut(hours, bins = bins, labels = labels, right = False, include_lowest = True)

In [8]:
# Extract day of week and month from date of incident
df['day_of_week'] = df['created_date'].dt.day_name()
df['month'] = df['created_date'].dt.month_name()

# Extract hours passed until resolution and resolution status
# Event is assumed to be resolved if the closed date is not the maximum date in 2024
df['resolution_time_hours'] = ((df['closed_date'] - df['created_date']).dt.total_seconds()
                                   / 3600).round().astype(int)
df['resolved'] = df['closed_date'] != pd.Timestamp("2024-12-31 23:59:59")

In [9]:
# Drop original time columns, as they will no longer be used
df = df.drop(['created_date', 'closed_date'], axis = 1)

In [10]:
# Check missingness of data again
df.isnull().sum()

agency_name                   0
complaint_type                0
location_type            401717
incident_zip              33781
city                     158600
borough                       0
latitude                  52836
longitude                 52836
time_of_day                   0
day_of_week                   0
month                         0
resolution_time_hours         0
resolved                      0
dtype: int64

### Check Target Variable (Agency Name)

In [11]:
# Only one instance of "3-1-1" and there are many categories with very few samples
df['agency_name'].value_counts()

agency_name
New York City Police Department                       1545786
Department of Housing Preservation and Development     710897
Department of Sanitation                               301513
Department of Transportation                           202607
Department of Environmental Protection                 189446
Department of Parks and Recreation                     126829
Department of Buildings                                 92369
Department of Health and Mental Hygiene                 68162
Department of Homeless Services                         51526
Taxi and Limousine Commission                           32598
Economic Development Corporation                        27491
Department of Consumer and Worker Protection            19326
Department of Education                                  1529
Office of Technology and Innovation                       154
3-1-1                                                       1
Name: count, dtype: int64

In [12]:
# Agencies which appear in low amounts of data will cause problems for modeling due to severe class imbalances
df['agency_name'].value_counts(normalize = True).round(4)

agency_name
New York City Police Department                       0.4587
Department of Housing Preservation and Development    0.2109
Department of Sanitation                              0.0895
Department of Transportation                          0.0601
Department of Environmental Protection                0.0562
Department of Parks and Recreation                    0.0376
Department of Buildings                               0.0274
Department of Health and Mental Hygiene               0.0202
Department of Homeless Services                       0.0153
Taxi and Limousine Commission                         0.0097
Economic Development Corporation                      0.0082
Department of Consumer and Worker Protection          0.0057
Department of Education                               0.0005
Office of Technology and Innovation                   0.0000
3-1-1                                                 0.0000
Name: proportion, dtype: float64

In [13]:
# We will wrap all of the agencies with less than 1% presence into a single "Other" category
df['agency_name'] = df['agency_name'].where(
    df['agency_name'].map(
        df['agency_name'].value_counts(normalize = True)
    ).gt(0.01), "Other"
)

In [14]:
# Checking agency name counts again - class imbalances are present but much improved
df['agency_name'].value_counts(normalize = True)

agency_name
New York City Police Department                       0.458658
Department of Housing Preservation and Development    0.210934
Department of Sanitation                              0.089464
Department of Transportation                          0.060117
Department of Environmental Protection                0.056212
Department of Parks and Recreation                    0.037632
Department of Buildings                               0.027407
Other                                                 0.024063
Department of Health and Mental Hygiene               0.020225
Department of Homeless Services                       0.015289
Name: proportion, dtype: float64

### Check Complaint Type

In [15]:
# There are many different complaint types, so it would be best to group these into categories of some sort
df['complaint_type'].unique()

array(['Noise - Commercial', 'Noise - Residential', 'Animal in a Park',
       'Rodent', 'Traffic Signal Condition', 'UNSANITARY CONDITION',
       'Noise - Park', 'Noise - Street/Sidewalk', 'Elevator', 'PLUMBING',
       'WATER LEAK', 'PAINT/PLASTER', 'Illegal Parking',
       'Non-Emergency Police Matter', 'Illegal Fireworks',
       'Noise - House of Worship', 'Blocked Driveway', 'Plumbing',
       'Taxi Complaint', 'Panhandling', 'Drug Activity',
       'Street Condition', 'Street Light Condition', 'Noise - Vehicle',
       'Water System', 'Drinking', 'For Hire Vehicle Report',
       'Root/Sewer/Sidewalk Condition', 'Encampment', 'Traffic',
       'Abandoned Vehicle', 'Noise', 'Animal-Abuse', 'Sewer',
       'Noise - Helicopter', 'Indoor Sewage', 'Bike/Roller/Skate Chronic',
       'Maintenance or Facility', 'ELECTRIC', 'APPLIANCE', 'DOOR/WINDOW',
       'GENERAL', 'Vendor Enforcement', 'Urinating in Public',
       'Unsanitary Animal Pvt Property', 'Real Time Enforcement',
      

In [16]:
# Creating a mapping for 12 different categories
# 1: Noise Complaints, 2: Animal-Relating Complaints, 3: Traffic & Parking
# 4: Sanitation & Maintenance, 5: Building & Facility Complaints, 6: Public Safety & Legal Issues
# 7. Environmental Complaints, 8: Property & Land Use, 9: Public Infrastructure & Transportation
# 10. Health & Food-Related, 11: Comunity Services & Outreach, 12: Other

# Mapping dictionary
complaint_mapping = {
    'Noise Complaints': [
        'Noise - Commercial', 'Noise - Residential', 'Noise - Park', 'Noise - Street/Sidewalk',
        'Noise - Vehicle', 'Noise - House of Worship', 'Noise - Helicopter', 'Noise'
    ],
    'Animal-Related Complaints': [
        'Animal in a Park', 'Rodent', 'Animal-Abuse', 'Unsanitary Animal Pvt Property', 
        'Unleashed Dog', 'Dead Animal', 'Animal Facility - No Permit', 
        'Unsanitary Animal Facility', 'Illegal Animal Sold', 'Illegal Animal Kept as Pet',
        'Harboring Bees/Wasps', 'Pet Shop', 'Unsanitary Pigeon Condition', 'Illegal Tree Damage'
    ],
    'Traffic & Parking': [
        'Bike/Roller/Skate Chronic', 'Derelict Vehicles', 'For Hire Vehicle Complaint',
        'Bike Rack Condition', 'AHV Inspection Unit', 'Stalled Sites',
        'Traffic Signal Condition', 'Illegal Parking', 'Blocked Driveway',
        'Taxi Complaint', 'For Hire Vehicle Report', 'Abandoned Vehicle', 'Street Condition',
        'Street Light Condition', 'Traffic', 'Taxi Compliment', 'Broken Parking Meter',
        'Street Sweeping Complaint', 'Street Sign - Damaged', 'Street Sign - Dangling',
        'Street Sign - Missing', 'Bus Stop Shelter Placement', 'Bus Stop Shelter Complaint',
        'Municipal Parking Facility', 'Highway Condition', 'Highway Sign - Damaged',
        'Highway Sign - Dangling', 'Highway Sign - Missing', 'Taxi Report', 'Green Taxi Report',
        'Green Taxi Complaint', 'Taxi Licensee Complaint', 'Dispatched Taxi Complaint',
        'FHV Licensee Complaint', 'Abandoned Bike', 'E-Scooter', 'Bike Rack', 'Tunnel Condition'
    ],
    'Building & Facility Complaints': [
        'UNSANITARY CONDITION', 'Sewer', 'Dirty Condition', 'Wood Pile Remaining', 'Mold',
        'OUTSIDE BUILDING', 'Peeling Paint', 'Non-Residential Heat', 'Elevator', 'Plumbing',
        'WATER LEAK', 'Boilers', 'APPLIANCE', 'DOOR/WINDOW', 'FLOORING/STAIRS', 'HEAT/HOT WATER',
        'Building/Use', 'Outdoor Dining', 'Building Condition', 'Maintenance or Facility',
        'Indoor Air Quality', 'Building Drinking Water Tank', 'Construction Safety Enforcement',
        'School Maintenance', 'Cooling Tower', 'Electrical', 'ELECTRIC', 'Cranes and Derricks',
        'Scaffold Safety', 'General Construction/Plumbing', 'Construction Lead Dust', 'Snow or Ice',
        'ELEVATOR', 'PLUMBING', 'PAINT/PLASTER', 'Indoor Sewage'
    ],
    'Public Safety & Legal Issues': [
        'Real Time Enforcement', 'Investigations and Discipline (IAD)',
        'Emergency Response Team (ERT)', 'Sanitation Worker or Vehicle Complaint',
        'Special Projects Inspection Team (SPIT)', 'Ferry Complaint', 'Lifeguard',
        'Building Marshals office', 'X-Ray Machine/Equipment', 'Non-Emergency Police Matter',
        'Drug Activity', 'Panhandling', 'Illegal Fireworks', 'Violation of Park Rules',
        'Encampment', 'Obstruction', 'Disorderly Youth', 'Smoking', 'Squeegee',
        'Public Payphone Complaint', 'Illegal Posting', 'Posting Advertisement',
        'SAFETY', 'Urinating in Public', 'Vendor Enforcement', 'Retailer Complaint',
        'Tobacco or Non-Tobacco Sale', 'BEST/Site Safety', 'Transfer Station Complaint',
        'Illegal Dumping', "Building Marshal's Office"
    ],
    'Environmental Complaints': [
        'Root/Sewer/Sidewalk Condition', 'Damaged Tree', 'Dead/Dying Tree',
        'Overgrown Tree/Branches', 'Sidewalk Condition', 'Curb Condition',
        'New Tree Request', 'Uprooted Stump', 'Lead', 'Bridge Condition',
        'Recycling Basket Complaint', 'Special Natural Area District (SNAD)',
        'Lot Condition', 'Air Quality', 'Water Conservation', 'Hazardous Materials',
        'Radioactive Material', 'Asbestos', 'Oil or Gas Spill', 'Mosquitoes',
        'Plant', 'DEP Highway Condition', 'Water System', 'Standing Water',
        'DEP Street Condition', 'DEP Sidewalk Condition', 'Industrial Waste'
    ],
    'Health & Food-Related': [
        'Food Poisoning', 'Food Establishment', 'Drinking', 'Drinking Water', 
        'Consumer Complaint', 'Building Drinking Water Tank', 'Mobile Food Vendor',
        'Dumpster Complaint', 'Water Quality', 'Institution Disposal Complaint',
        'Residential Disposal Complaint', 'Commercial Disposal Complaint'
    ],
    'Community Services & Outreach': [
        'Litter Basket Request', 'Litter Basket Complaint', 'Adopt-A-Basket', 'Bench',
        'Wayfinding', 'Seasonal Collection', 'DSNY Internal', 'SNW', 'Homeless Person Assistance',
        'Lost Property', 'Found Property', 'Consumer Complaint', 'Borough Office',
        'Special Operations', 'Executive Inspections', 'Sustainability Enforcement',
        'Dept of Investigations', 'Public Toilet', 'LinkNYC', 'Ferry Inquiry', 'Beach/Pool/Sauna Complaint'
    ]
}

# Invert the dictionary for mapping
inverse_mapping = {}
for category, types in complaint_mapping.items():
    for complaint in types:
        inverse_mapping[complaint] = category

# Map the complaint_type to categories, mapping unspecified complaint types to "Other"
df['complaint_type'] = df['complaint_type'].map(inverse_mapping).fillna('Other')

In [17]:
# Checking the counts for the new categories, these categories are more reasonable
df['complaint_type'].value_counts(normalize = True)

complaint_type
Traffic & Parking                 0.299834
Building & Facility Complaints    0.261038
Noise Complaints                  0.222360
Public Safety & Legal Issues      0.071356
Environmental Complaints          0.068017
Other                             0.024805
Animal-Related Complaints         0.022657
Community Services & Outreach     0.021199
Health & Food-Related             0.008735
Name: proportion, dtype: float64

### Location Type

In [18]:
# There are missing values in this column
df['location_type'].isnull().sum()

401717

In [19]:
# Like with the complaint type, there are a lot of unique location types
# It would be helpful to put these into categories
df['location_type'].unique()

array(['Store/Commercial', 'Residential Building/House', 'Park',
       'Commercial Building', nan, 'Club/Bar/Restaurant',
       'RESIDENTIAL BUILDING', 'Park/Playground', 'Street/Sidewalk',
       'House of Worship', 'Street', 'Subway', 'Other', 'Hallway',
       'Lobby', 'Highway', 'Above Address', '1-2 Family Dwelling',
       'Common Area', '3+ Family Apartment Building',
       'Other (Explain Below)', '3+ Family Apt. Building', 'Sidewalk',
       'Bridge', '3+ Family Mixed Use Building', 'Taxi', 'Curb',
       'Restaurant/Bar/Deli/Bakery', 'Public/Unfenced Area',
       'Street/Curbside', 'Comercial', 'Yard', 'Alley', 'Public Garden',
       'Construction Site', 'Intersection', 'Gutter', 'Business',
       'Residence', 'Vacant Lot', 'House and Store', 'Food Cart Vendor',
       'Traffic Island or Median', 'Residential Building', 'Mixed Use',
       'Residential', 'Bus Stop Shelter', 'Kennel/Animal Shelter',
       '1-2 Family Mixed Use Building', 'Retail Store',
       'Mobile F

In [20]:
# Creating a mapping for 6 different categories
# 1: Street, 2: Residential, 3: Commercial,
# 4: Restaurant, 5: Outdoor, 6: Other

# Mapping dictionary
location_mapping = {
    "Street": [
        'Street/Sidewalk', 'Street', 'Sidewalk', 'Highway', 'Intersection',
        'Alley', 'Curb', 'Street/Curbside', 'Taxi', 'Bus Stop Shelter',
        'Bike Lane', 'Vehicle Lane', 'Crosswalk', 'Speed Reducer',
        'Catch Basin/Sewer', 'Roadway Tunnel', 'Overpass', 'Public Stairs',
        'Street Area', 'Bridge', 'Gutter', 'Subway', 'Subway Station',
        'Street Fair Vendor'
    ],
    "Residential": [
        'Residential Building/House', 'RESIDENTIAL BUILDING', '1-2 Family Dwelling',
        '3+ Family Apartment Building', '3+ Family Apt. Building',
        '3+ Family Mixed Use Building', 'House and Store', 'Residence',
        'Residential Building', 'Residential', 'Private Residence',
        '1-3 Family Dwelling', '1-3 Family Mixed Use Building',
        '1-2 Family Mixed Use Building', 'Residential Property', 'Loft Residence',
        'Single Room Occupancy (SRO)', 'Nursing Home'
    ],
    "Commercial": [
        'Store/Commercial', 'Commercial Building', 'Comercial', 'Business',
        'Retail Store', 'Commercial', 'Commercial Property', 'Store',
        'Tire Shop', 'Groomer', 'Tanning Salon', 'Steam Room', 'Spa Pool',
        "Doctor's Office", 'Building (Non-Residential)', 'Office Building',
        'Medical Facility', 'Mixed Use Building', 'Sports Arena', 'Theater'
    ],
    "Restaurant": [
        'Club/Bar/Restaurant', 'Restaurant/Bar/Deli/Bakery', 'Food Cart Vendor',
        'Mobile Food Vendor', 'Food Establishment or Vendor', 'Restaurant',
        'Catering Service', 'Green Cart Vendor', 'Street Vendor',
        'Permanent Food Stand', 'Soup Kitchen', 'Cafeteria - Private',
        'Cafeteria - Private School', 'Cafeteria - Public School', 'Cafeteria - College/University'
    ],
    "Outdoor": [
        'Park', 'Park/Playground', 'Public/Unfenced Area', 'Yard',
        'Construction Site', 'Vacant Lot', 'Lot', 'Roof', 'Beach',
        'Ground', 'Pier', 'Wooded Area', 'Swamp or Pond', 'Horse Stable',
        'Petting Zoo/Animal Exhibit', 'Pool', 'Vacant Lot/Property', 'Parking Lot/Garage',
        'Parking Lot', 'Garage', 'Terminal', 'Ferry', 'Airport or Train/Bus Station',
        'Public Garden', 'Public Park/Garden', 'Public Garden/Park', 'Public Plaza',
        'Traffic Island or Median'
    ],
    "Other": [
        None, 'Other', 'House of Worship', 'Hallway', 'Lobby', 'Above Address',
        'Common Area', 'Other (Explain Below)', 'Kennel/Animal Shelter',
        'Inside', 'Vacant Building', 'Private Property', 'Abandoned Building',
        'Building Entrance', 'Building Condition', 'Building', 'Non-Profit',
        'Loft Building - Common Areas', 'Hospital', 'Cemetery', 'Senior Center',
        'Homeless Shelter', 'Sauna', 'Stairwell', 'Correctional Facility - City',
        'Correctional Facility - State', 'Mixed Use', 'Other (explain in Complaint Details)',
        'Government Building', 'School', 'School/Pre-School', 'Day Care/Nursery', 'School Safety Zone',
        'School - K-12 Public', 'School - K-12 Private', 'School - College/University'
    ]
}

# Invert the dictionary for mapping
inverse_mapping = {}
for category, types in location_mapping.items():
    for complaint in types:
        inverse_mapping[complaint] = category

# Map the location_type to categories, mapping unspecified complaint types to "Other"
df['location_type'] = df['location_type'].map(inverse_mapping).fillna('Other')

In [21]:
# Checking the counts for the new categories, these categories are more reasonable
df['location_type'].value_counts(normalize = True)

location_type
Street         0.463156
Residential    0.347639
Other          0.134447
Commercial     0.024549
Outdoor        0.018521
Restaurant     0.011688
Name: proportion, dtype: float64

### Location Data (City, Borough, Latitude/Longitude, and ZIP Code)

In [22]:
# There are a lot of values that need to be cleaned up, including null values
df['city'].unique()

array(['NEW YORK', 'STATEN ISLAND', 'BRONX', 'QUEENS', 'BROOKLYN',
       'OAKLAND GARDENS', nan, 'CORONA', 'JAMAICA', 'ASTORIA',
       'FOREST HILLS', 'FAR ROCKAWAY', 'ELMHURST', 'JACKSON HEIGHTS',
       'OZONE PARK', 'RIDGEWOOD', 'REGO PARK', 'COLLEGE POINT',
       'SOUTH RICHMOND HILL', 'EAST ELMHURST', 'MANHATTAN', 'WOODHAVEN',
       'SOUTH OZONE PARK', 'MASPETH', 'WOODSIDE', 'FLUSHING',
       'LITTLE NECK', 'SPRINGFIELD GARDENS', 'KEW GARDENS', 'WHITESTONE',
       'MIDDLE VILLAGE', 'BELLEROSE', 'FLORAL PARK', 'LONG ISLAND CITY',
       'ROSEDALE', 'FRESH MEADOWS', 'BAYSIDE', 'QUEENS VILLAGE',
       'HOWARD BEACH', 'SAINT ALBANS', 'SUNNYSIDE', 'RICHMOND HILL',
       'HOLLIS', 'ARVERNE', 'CAMBRIA HEIGHTS', 'GLEN OAKS',
       'NEW HYDE PARK', 'Nesconset', 'bronx', 'Staten Island', 'KATY',
       'Elmhurst', 'Manhattan', 'brooklyn', 'BREEZY POINT', 'Bentonville',
       'Brooklyn', 'New York', 'LAGUARDIA AIRPORT', 'Yonkers',
       'West Hempstead', 'La Grange', 'hicksville',

In [23]:
# Some of these locations are not in NYC, so let's preserve only locations in NYC and unknowns
nyc_areas = [
    'arverne', 'astoria', 'bayside', 'bellerose', 'breezy point', 'bronx',
    'brooklyn', 'cambria heights', 'college point', 'corona', 'east elmhurst',
    'elmhurst', 'far rockaway', 'flushing', 'forest hills', 'fresh meadows',
    'glen oaks', 'glendale queens', 'hollis', 'howard beach', 'jackson heights',
    'jamaica', 'jamaica queens', 'kew gardens', 'laguardia airport',
    'little neck', 'long island city', 'manhattan', 'maspeth', 'middle village',
    'new york', 'oakland gardens', 'ozone park', 'queens', 'queens village',
    'rego park', 'ridgewood', 'richmond hill', 'rosedale', 'saint albans',
    'south ozone park', 'south richmond hill', 'springfield boulevard',
    'springfield gardens', 'staten island', 'sunnyside', 'woodhaven', 'woodside',
    'whitestone', 'unknown'
]

# Convert city names to lowercase to normalize
df['city'] = df['city'].str.lower()

# Print number of dropped rows
print(f'Number of dropped rows: {len(df[(~df["city"].isin(nyc_areas)) & (df["city"].notna())])}')

# Drop all non-null rows outside of NYC
df = df[(df['city'].isin(nyc_areas)) | (df['city'].isna())]

Number of dropped rows: 1613


In [24]:
# The easiest way to impute the city is to use the borough name
# Checking that there are no rows where the city and borough are both null
df[(df['city'].isna()) & (df['borough'].isna())]

,agency_name,complaint_type,location_type,incident_zip,city,borough,latitude,longitude,time_of_day,day_of_week,month,resolution_time_hours,resolved


In [25]:
# Impute missing values with borough names
df['city'] = df['city'].fillna(df['borough'].str.lower())

In [26]:
# Very few of the cities are prevalent enough to be worth using
# The most common "cities" are just borough names
df['city'].value_counts(normalize = True)[df['city'].value_counts(normalize = True).gt(0.01)].round(4)

city
brooklyn         0.3024
bronx            0.2142
new york         0.1895
staten island    0.0351
queens           0.0253
jamaica          0.0233
manhattan        0.0188
astoria          0.0170
flushing         0.0148
ridgewood        0.0115
Name: proportion, dtype: float64

In [27]:
# Even the "new york" designation is equivalent to Manhattan
df[df['city'] == 'new york'].borough.value_counts()

borough
MANHATTAN      638359
Unspecified        32
Name: count, dtype: int64

In [28]:
# Since the city is no longer useful, drop the city column
df = df.drop(['city'], axis = 1)

In [29]:
# Check the unique borough names, no null values and each value is relevant, just need to normalize
df['borough'].unique()

array(['MANHATTAN', 'STATEN ISLAND', 'BRONX', 'QUEENS', 'BROOKLYN',
       'Unspecified'], dtype=object)

In [30]:
# Convert all values in borough to uppercase
df['borough'] = df['borough'].str.upper()

In [31]:
# There are a number of missing zip codes
df.incident_zip.isna().sum()

33781

In [32]:
# There are a lot of ZIP codes, let's check to make sure each are in NYC
df['incident_zip'].unique()

array(['10031', '10013', '10314', '10456', '10128', '11421', '10004',
       '10472', '11208', '11364', '10025', '10457', '11236', '11435',
       '11368', '11432', '11106', '10467', '11375', '10305', '11694',
       '11223', '10016', '11207', '11373', '11224', '11230', '11204',
       '11210', '11239', '10012', '11233', '11372', '11219', '11417',
       '11232', '11220', '11385', '10466', '10462', '10454', '11237',
       '11374', '10453', '10040', '11356', '10465', '10312', '11238',
       '11221', '11691', '10033', '10022', '10038', '11419', '10452',
       '10461', '11369', '10028', '10029', '11214', '11206', '10027',
       '11226', '10302', '11215', '10034', nan, '10455', '10030', '10007',
       '11235', '11358', '11201', '10009', '10001', '11420', '10039',
       '11212', '11378', '11377', '10003', '10459', '11216', '11355',
       '10019', '11222', '10463', '10024', '10002', '10304', '11211',
       '11370', '11365', '10451', '10310', '11218', '11234', '11205',
       '11362',

In [33]:
# ZIP codes from Zillow: https://www.zillow.com/browse/homes/ny/new-york-county/
nyc_zip = [
    "10001", "10002", "10003", "10004", "10005", "10006", "10007", "10008", "10009",
    "10010", "10011", "10012", "10013", "10014", "10015", "10016", "10017", "10018",
    "10019", "10020", "10021", "10022", "10023", "10024", "10025", "10026", "10027",
    "10028", "10029", "10030", "10031", "10032", "10033", "10034", "10035", "10036",
    "10037", "10038", "10039", "10040", "10041", "10043", "10044", "10045", "10046",
    "10047", "10048", "10049", "10050", "10051", "10052", "10053", "10054", "10055",
    "10056", "10057", "10058", "10059", "10060", "10061", "10062", "10063", "10064",
    "10065", "10066", "10067", "10068", "10069", "10070", "10071", "10072", "10073",
    "10074", "10075", "10076", "10077", "10078", "10079", "10080", "10081", "10082",
    "10083", "10084", "10085", "10086", "10087", "10088", "10089", "10090", "10091",
    "10092", "10093", "10094", "10095", "10096", "10097", "10098", "10099", "10101",
    "10102", "10103", "10104", "10105", "10106", "10107", "10108", "10109", "10110",
    "10111", "10112", "10113", "10114", "10115", "10116", "10117", "10118", "10119",
    "10120", "10121", "10122", "10123", "10124", "10125", "10126", "10127", "10128",
    "10129", "10130", "10131", "10132", "10133", "10134", "10135", "10136", "10137",
    "10138", "10139", "10140", "10141", "10142", "10143", "10144", "10145", "10146",
    "10147", "10148", "10149", "10150", "10151", "10152", "10153", "10154", "10155",
    "10156", "10157", "10158", "10159", "10160", "10161", "10162", "10163", "10164",
    "10165", "10166", "10167", "10168", "10169", "10170", "10171", "10172", "10173",
    "10174", "10175", "10176", "10177", "10178", "10179", "10180", "10181", "10182",
    "10183", "10184", "10185", "10186", "10187", "10188", "10189", "10190", "10191",
    "10192", "10193", "10194", "10195", "10196", "10197", "10198", "10199", "10201",
    "10202", "10203", "10204", "10205", "10206", "10207", "10208", "10209", "10210",
    "10211", "10212", "10213", "10214", "10215", "10216", "10217", "10218", "10219",
    "10220", "10221", "10222", "10223", "10224", "10225", "10226", "10227", "10228",
    "10229", "10230", "10231", "10232", "10233", "10234", "10235", "10236", "10237",
    "10238", "10239", "10240", "10241", "10242", "10243", "10244", "10245", "10246",
    "10247", "10248", "10249", "10250", "10251", "10252", "10253", "10254", "10255",
    "10256", "10257", "10258", "10259", "10260", "10261", "10262", "10263", "10264",
    "10265", "10266", "10267", "10268", "10269", "10270", "10271", "10272", "10273",
    "10274", "10275", "10276", "10277", "10278", "10279", "10280", "10281", "10282",
    "10283", "10284", "10285", "10286"
]

# Convert values in the column to None if not in the nyc_zip list
df['incident_zip'] = df['incident_zip'].apply(lambda x: x if x in nyc_zip else None)

In [34]:
# Want to impute missing ZIP codes using latitude and longitude
# Using GeoPandas and ZCTA shapefile from US Census: https://www2.census.gov/geo/tiger/TIGER2023/ZCTA520/
# Create Point geometry column
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis = 1)
gdf_points = gpd.GeoDataFrame(df, geometry = 'geometry', crs = 'EPSG:4326')

# Load the ZCTA shapefile
zcta = gpd.read_file('../data/raw/tl_2023_us_zcta520.shp')
zcta = zcta.to_crs('EPSG:4326')  # Ensure same coordinate system

# Spatial join: assign ZIP code to each point
gdf_joined = gpd.sjoin(gdf_points, zcta[['ZCTA5CE20', 'geometry']], how = 'left', predicate = 'within')

# Fill missing ZIP codes only
df['incident_zip'] = df['incident_zip'].fillna(gdf_joined['ZCTA5CE20'])

In [35]:
# For the remaining missing ZIP codes, impute using the mode from each borough (as borough is non-missing)
# Compute mode ZIP for each borough
mode_zip = df.dropna(subset = ['incident_zip']).groupby('borough')['incident_zip'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)

# Create a mask for missing ZIPs
missing_mask = df['incident_zip'].isna()

# Fill missing ZIPs based on borough
df.loc[missing_mask, 'incident_zip'] = df.loc[missing_mask, 'borough'].map(mode_zip)

In [36]:
# Impute missing latitude and longitude values using the modes from the now non-missing ZIP column
# Calculate mode latitue/longitude per ZIP
lat_mode = df.dropna(subset = ['latitude']).groupby('incident_zip')['latitude'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)
lon_mode = df.dropna(subset = ['longitude']).groupby('incident_zip')['longitude'].agg(
    lambda x: x.mode().iloc[0] if not x.mode().empty else None
)

# Create masks for missing latitude/longitude
lat_missing = df['latitude'].isna()
lon_missing = df['longitude'].isna()

# Fill missing values using the ZIP mode mappings
df.loc[lat_missing, 'latitude'] = df.loc[lat_missing, 'incident_zip'].map(lat_mode)
df.loc[lon_missing, 'longitude'] = df.loc[lon_missing, 'incident_zip'].map(lon_mode)

### Final Checks and Save in Preparation for EDA

In [38]:
# Check for missing values - should be none left
df.isnull().sum()

agency_name              0
complaint_type           0
location_type            0
incident_zip             0
borough                  0
latitude                 0
longitude                0
time_of_day              0
day_of_week              0
month                    0
resolution_time_hours    0
resolved                 0
geometry                 0
dtype: int64

In [40]:
# Check data types, they seem appropriate
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3368621 entries, 64 to 3458310
Data columns (total 13 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   agency_name            object  
 1   complaint_type         object  
 2   location_type          object  
 3   incident_zip           object  
 4   borough                object  
 5   latitude               float64 
 6   longitude              float64 
 7   time_of_day            category
 8   day_of_week            object  
 9   month                  object  
 10  resolution_time_hours  int64   
 11  resolved               bool    
 12  geometry               object  
dtypes: bool(1), category(1), float64(2), int64(1), object(8)
memory usage: 314.8+ MB


In [41]:
# Save dataframe for further use in EDA without having to reclean the data
df.to_csv('../data/cleaned/311_Data_Cleaned_EDA.csv.bz2',
          compression = {'method': 'bz2', 'compresslevel': 9}, index = False)

## EDA